In [1]:
import dvc.api
import pandas as pd

In [2]:
import sys 
sys.path.append('../../')

In [3]:
from main_multitask import LLM_Multitask

In [3]:
model = LLM_Multitask.load_from_checkpoint('../../models/product_title_multitask/version_1/epoch=0-step=349636.ckpt', 
    label_map_file='../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt')

INFO:root:Unused kwargs when getting t5-base: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-last', 'add_simcse': True, 'label_map_file': '../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt', 'label_weight_type': 'ancestor-high', 'additional_tricks': None, 'multitask_names': ['clm_ali_title2tax', 'dlm_wish_title', 'emb_all_title2tax', 'seqclf_ali_title2tax'], 'auto_task_weight': False}
/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instan

In [4]:
model.eval()
tmp = 2

# useful

In [51]:
df_sam = pd.read_csv('../../tmp_test_value_list.csv')
df_sam['amazon_l1_flattype'] = (df_sam.L1_source + " > " + df_sam.product_type).str.lower()
len(set(df_sam.L1_source)), len(set(df_sam.product_type)), len(set(df_sam.amazon_l1_flattype))

(37, 984, 995)

In [59]:
results = {i: [] for i in set(df_sam['amazon_l1_flattype'])}

In [60]:
# seqclf-based 0-shot categorize ood taxonomy from google shopping
label_map_rev = {model.label_map[i]: i for i in model.label_map}


In [61]:
from tqdm import tqdm

In [62]:
for t in tqdm(results):
    inputs = model.tokenizer(f"Classify taxonomy: {t}", 
        return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    hidden_states = model.get_hidden_states(
        input_ids = input_ids, 
        attention_mask = attention_mask
    )
    logits = model.clf_head(hidden_states)
    prediction = logits
    probs = prediction.sigmoid()
    top_probs, top_pred_indices = probs.topk(probs.size(1))
    assert prediction.size(1) == len(model.label_map)
    results[t] = [(label_map_rev[i], probs[0][i].item()) for i in top_pred_indices.detach().cpu().numpy().reshape(-1) if label_map_rev[i].count('>') == 1]


100%|██████████| 995/995 [13:50<00:00,  1.20it/s] 


In [77]:
df_sam['top_prediced_wish_l2s'] = df_sam['amazon_l1_flattype'].apply(lambda x: results[x][:25])

In [79]:
df_sam.to_csv('../../tmp_test_value_list_withpredwishl2s.csv', index=False)

In [4]:
import pandas as pd
df_sam = pd.read_csv('../../tmp_test_value_list_withpredwishl2s.csv')

In [7]:
df_sam.groupby(['product_type', 'L1_source']).agg({"top_prediced_wish_l2s": "first"}).reset_index().to_csv(
    '../../tmp_test_value_list_withpredwishl2s_groupby.csv', index=False)

In [6]:
df_sam.sample(5).to_dict('records')

[{'attribute': 'Maximum Weight Recommendation Unit Of Measure',
  'product_type': 'toybuildingblock',
  'L1_source': 'Toy',
  'attr_values': "['LB', 'Tons', 'OZ', 'KG', 'MG', 'GR', 'Hundredths Pounds']",
  'amazon_l1_flattype': 'toy > toybuildingblock',
  'top_prediced_wish_l2s': "[('toys & hobbies > classic toys', 0.12237467616796494), ('sports > team sports', 0.04356149211525917), ('sports > fitness & body building', 0.042670391499996185), ('education & office supplies > books & magazines', 0.037974338978528976), ('toys & hobbies > dolls & accessories', 0.03540969640016556), ('toys & hobbies > learning & education', 0.034304190427064896), ('entertainment > musical instruments', 0.026826132088899612), ('toys & hobbies > stuffed animals & plush toys', 0.02505052275955677), ('sports > sports clothing', 0.019445398822426796), ('toys & hobbies > baby & toddler toys', 0.014485400170087814), ('toys & hobbies > action & toy figures', 0.013625518418848515), ('sports > sneakers', 0.01062208972

In [74]:
results['consumerelectronics > tuner'][:10]

[('automobiles & motorcycles > car electronics', 0.08472103625535965),
 ('consumer electronics > portable audio & video', 0.05618169158697128),
 ('entertainment > musical instruments', 0.0555446483194828),
 ('tools > measurement & analysis instruments', 0.05305119603872299),
 ('home improvement > home appliances', 0.04363132640719414),
 ('automobiles & motorcycles > car repair tools', 0.03162699565291405),
 ('consumer electronics > home audio & video', 0.031125836074352264),
 ('automobiles & motorcycles > auto replacement parts', 0.021887294948101044),
 ('electronic components & supplies > passive components',
  0.011953347362577915),
 ('computer & office > networking', 0.009184069000184536)]